<a href="https://colab.research.google.com/github/Zozz98/Finance-Economics/blob/main/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import plotly.express as px
import yfinance as yf
import pandas_datareader as pdr
import datetime as dt

from sklearn.cluster import KMeans

# GET DATA FROM WIKIPEDIA

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_list = pd.read_html(url)
sp500_list = sp500_list[0]
sp500_list = sp500_list.drop(columns=['GICS Sub-Industry','Headquarters Location', 'Date added','CIK','Founded'], axis=1)
sp500_list.rename(columns={'GICS Sector':'Sector'},inplace=True)

# LOAD CLOSE PRICES FROM WIKIPEDIA DATA

In [3]:
end = dt.datetime.now()
start = end - dt.timedelta(days=252)

ticker_list = [i for i in sp500_list['Symbol']]
stocks = yf.download(ticker_list, start=start, end=end)['Close']
stocks.drop(columns=['BF.B','BRK.B'],inplace=True)

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BRK.B: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


# LOG RETURN AND SET COLUMNS

In [4]:
log_return = np.log(stocks / stocks.shift(1)).replace(np.nan, 0)

annual_return = log_return.mean() * 252
annual_volatility = log_return.std() * np.sqrt(252)

stock_data = pd.DataFrame()
stock_data['Symbol'] = stocks.columns
stock_data['Security'] = sp500_list['Security']
stock_data['Sector'] = sp500_list['Sector']
stock_data['Return'] = list(round(annual_return, 3))
stock_data['Volatility'] = list(round(annual_volatility, 3))
stock_data['Ratio'] = list(round((annual_return / annual_volatility), 3))

# VOLATILITY-RETURN SCATTER PLOT

In [7]:
px.scatter(stock_data, 
           x='Volatility', 
           y='Return', 
           hover_data=['Sector'],
           hover_name='Symbol', 
           title='S&P 500 stocks by Volatility/Return', 
           color='Ratio',
           width=750,
           height=500,
           color_continuous_scale='rainbow').update_layout(title_x=0.5)

# FIND NUMBER OF CLUSTERS WITH ELBOW-METHOD

In [8]:
X = stock_data[['Return', 'Volatility']]
inertia_list = []
for k in range(2,16):
    kmeans = KMeans(n_clusters=k, n_init='auto')
    kmeans.fit(X)
    inertia_list.append(kmeans.inertia_)

px.line(inertia_list, 
        title='Elbow Curve',
        width=750,
        height=500,
        labels={
            "index":"Number of Clusters",
            "value":"Sum of Squared Error"
        }).update_layout(showlegend=False, title_x=0.5)

# KMEAN MODEL FIT

In [9]:
kmeans = KMeans(n_clusters=2, n_init='auto').fit(X)
labels = kmeans.labels_
stock_data['Cluster Label'] = labels

# PLOT STOCKS BY CLUSTERS

In [10]:
px.scatter(X, 
           x='Volatility', 
           y='Return', 
           color=labels,
           title='KMeans Clusters',
           width=750,
           height=500,
           color_continuous_scale='rainbow').update_layout(title_x=0.5)